Импортирование модели:

In [ ]:
from store.models import Product #

# Запросы на получение данных

Получение всех записей:

In [ ]:
Product.objects.all()

Получить запись со значением первичного ключа равного единице (при этом указание имени первичного ключа не требуется):

In [ ]:
Product.objects.get(pk=1)

Если нет записи с таким значением первичного ключа, будет выдано исключение `ObjectDoesNotExist`:

In [ ]:
from django.core.exceptions import ObjectDoesNotExist

Product.objects.get(pk=0)

В данном случае будет возвращен набор записей. Если записей нет -- набор будет пустым:

In [ ]:
Product.objects.filter(pk=0)

Если использовать метод `.first()` то он вернёт первый объект при наличии или значение `None` если объект не найден:

In [ ]:
Product.objects.filter(pk=0).first()

# Отбор данных

In [ ]:
Product.objects.filter(unit_price=20) # цена равна 20
Product.objects.filter(unit_price__in=(1, 2, 5)) # цена равна 1 2 или 5
Product.objects.filter(unit_price__gt=20) # цена больше 20
Product.objects.filter(unit_price__range=(1, 5)) # цена от 1 до 5
Product.objects.filter(title__icontains="coffee") # содержит строку "coffee" без учета регистра
Product.objects.filter(title__istartswith="coffee") # начинается со строки "coffee" без учета регистра
Product.objects.filter(last_update__year=2020) # год последнего обновления 2020

In [ ]:
from django.db.models import Q, F

Product.objects.filter(unit_price__gt=20, title__istartswith="coffee") # стоимость больше 20, заголовок начинается с "coffee"
Product.objects.filter(Q(unit_price__gt=80) | Q(title__istartswith="coffee")) # стоимость больше 80 ИЛИ заголовок начинается с "coffee"
Product.objects.filter(~Q(unit_price__gt=80) & Q(title__istartswith="coffee")) # стоимость меньше или равна 80 и заголовок начинается с "coffee"

Product.objects.filter(unit_price__gt=2 * F("inventory")) # стоимость больше, чем запас склада умноженный на 2

# Сортировка данных

In [ ]:
Product.objects.order_by("unit_price") # сортировка по не убыванию цены
Product.objects.order_by("-unit_price") # сортировка по не возрастанию цены
Product.objects.order_by("-inventory", "title") # сортировка по не возрастанию запаса на складе, при равном значении номеров -- по названию

# получение самого дорогого товара (вернут один объект)
Product.objects.order_by("-unit_price")[0]
Product.objects.latest("unit_price")

# получение самого дешевого товара (вернут один объект)
Product.objects.order_by("unit_price")[0]
Product.objects.earliest("unit_price")

# вывод первых десяти записей
Product.objects.order_by("unit_price")[:10]
# вывод следующих десяти записей
Product.objects.order_by("unit_price")[10:20]

# Выбор полей для формирования результата

In [ ]:
# получение набора объектов с id и значением заголовка из таблицы collection_title
Product.objects.values('id', 'collection__title') # вернёт набор словарей

In [ ]:
Product.objects.values_list('id', 'collection__title') # вернёт набор кортежей

In [ ]:
from store.models import OrderItem

OrderItem.objects.values('product_id').distinct() # получение уникальных наборов данных из словарей с ключом product_id

In [ ]:
ids = OrderItem.objects.values('product_id').distinct()
Product.objects.filter(id__in=ids)

In [ ]:
Product.objects.only('title') # работает как values и возвращает объекты, но если запрашиваемая страница будет требовать значений других полей, последует множество SELECT запросов с целью найти значение тех полей

In [ ]:
Product.objects.only('title') # работает как only только перечисляются поля, которые не должны попасть в создаваемый объект

In [ ]:
product_join_collection = Product.objects.select_related('collection').all()

# Вычисляемые столбцы

In [ ]:
from django.db.models import Value

Product.objects.annotate(is_new=Value(False)) # добавить столбец is_new со значением False

In [ ]:
Product.objects.annotate(new_price=F("unit_price") * 2) # добавить вычисляемый столбец со значением цены, умноженной на 2

In [ ]:
# использование функций, предоставляемых базой данных
from django.db.models import Func

from store.models import Customer

Customer.objects.annotate(
    full_name=Func(F('first_name'), Value(' '), F('last_name'), function='CONCAT')
)

In [ ]:
# аналогично, но короче
from django.db.models.functions import Concat

Customer.objects.annotate(
    full_name=Concat(F('first_name'), Value(' '), F('last_name'))
)

# Группировка

In [ ]:
from django.db.models import Count

Customer.objects.annotate(
    order_count=Count('order')
)

In [ ]:
from django.db.models import ExpressionWrapper, DecimalField

discounted_price = ExpressionWrapper(F('unit_price') * 0.8,
                                     output_field=DecimalField())